<a href="https://colab.research.google.com/github/gretiere545/tradunion/blob/master/corpus_glossary_builder_v2_blue_grey_brown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création d'un glossaire alphabétique des Corpus ASAMLA

## Chargement des Corpus

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-

!pip install fpdf
!pip install arabic_reshaper
# https://pyfpdfbook.wordpress.com/2015/03/22/putting-two-adjacent-multicell-blocks/
import pandas as pd
import numpy as np
from fpdf import FPDF
import arabic_reshaper
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
!export PYTHONIOENCODING=utf8
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40722 sha256=9a70e276c78f02e201172b6d1baca7d663c090fb565c1b1472e0157304d4b9c6
  Stored in directory: /root/.cache/pip/wheels/9a/e9/77/4554ff5c99bc3f487c8d69620d8c41d99d54e9c54ab20ef4c9
Successfully built fpdf
Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [21]:
uploaded_cover = files.upload()

Saving marcel-strauss-3Zlg1lgCu4E-unsplash.jpg to marcel-strauss-3Zlg1lgCu4E-unsplash.jpg


In [3]:
# cyrillique
!apt-get update -qq
!apt-get install -y fonts-dejavu-core -qq

# amharique
!apt-get update
!apt-get install fonts-noto




Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-1_all.deb ...
Unpacking fonts-dejavu-core (2.37-1) ...
Setting up fonts-dejavu-core (2.37-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9

In [ ]:
# arabic
!apt-get update
!apt-get install lemonada


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [4]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
#ouverture de l'onglet Corpus dans un DF
t_corpus = wb_central.worksheet('med_vac_synthese')
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
df = df_corpus[['uid','expression','rus','index']]



In [5]:
def charge_corpus_langue(df_corpus, langue):
  #ouverture de l'onglet Corpus dans un DF
  df = df_corpus[['uid','expression', langue,'index']]
  return df


In [6]:
vk_lang_dict=[
{'language':'Arabe','trigramme':'ams'},
{'language':'Turc','trigramme':'tur'},
{'language':'Russe','trigramme':'rus'},
{'language':'Ukrainien','trigramme':'ukr'},
{'language':'Roumain','trigramme':'rou'},
{'language':'Hongrois','trigramme':'hun'},
{'language':'Tigrinya','trigramme':'tig'},
{'language':'Albanais','trigramme':'alb'},
{'language':'Géorgien','trigramme':'geo'},
{'language':'Arménien','trigramme':'arm'},
{'language':'Dari','trigramme':'dar'},
{'language':'Pashto','trigramme':'pst'},
{'language':'Fârsi','trigramme':'prs'},
{'language':'Azéri','trigramme':'aze'},
{'language':'Espagnol','trigramme':'esp'},
{'language':'Amharique','trigramme':'amh'},
{'language':'Peul','trigramme':'fma'},
]

In [30]:
class PDF(FPDF):
  def __init__(self):
    super().__init__()
    self.WIDTH = 210
    self.HEIGHT = 297
    self.format = 'A4'
    self.unit = 'in'
    self.set_margins(20.0, 10.0, 20.0)
    self.color_1 = (89, 80, 72) # marron
    self.color_2 = (197, 217, 213) # blue pale   
    self.color_3 = ( 74, 128, 140) # bleu ocean
    self.color_4 = (142, 186, 191) # bleu ciel        
    self.logo_1 = './logo_asamla.jpg'
    self.font_1 = ('Montserrat-Medium', '', 11)
    self.font_2 = ('Montserrat-Medium', '', 24)  
    self.font_cover_1 = ('BebasNeue-Regular', '', 60)  
    self.title_1 = 'Corpus Médical'
    self.title_2 = 'Vaccination'
    self.cover_1 = 'marcel-strauss-3Zlg1lgCu4E-unsplash.jpg'

  def header(self):
    # Custom logo and positioning
    # Create an `assets` folder and put any wide and short image inside
    # Name the image `logo.png`
    self.image(self.logo_1, 10, 8, 33)
    self.set_text_color (*self.color_1)
    self.set_font(*self.font_1)
    self.cell(self.WIDTH - 80)
    self.cell(50, 1, self.title_1, 0, 0, 'R')
    self.ln(5)      
    self.cell(self.WIDTH - 80)
    self.set_text_color (*self.color_1)
    self.cell(50, 1, 'Français-'+self.local_language, 0, 0, 'R')      
    new_y = self.get_y()+7
    self.line(0, new_y, 210, new_y)

  def print_page(self, images):
    # Generates the report
    self.add_page()  

  def set_langue(self, langue):
    self.local_language = langue

  def add_fonts(self):
    self.add_font('DejaVuSans', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
    self.add_font('DejaVuSans-Bold', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)    
    self.add_font('NotoSerifEthiopic', '', '/usr/share/fonts/truetype/noto/NotoSerifEthiopic-Regular.ttf', uni=True)
    self.add_font('NotoSansArabic', '', '/usr/share/fonts/truetype/noto/NotoSansArabic-Regular.ttf', uni=True) 
    self.add_font('BebasNeue-Regular','', 'BebasNeue-Regular.ttf', uni=True)   
    self.add_font('Slackey-Regular','', 'Slackey-Regular.ttf', uni=True)  
    self.add_font('Montserrat-Medium','','Montserrat-Medium.ttf', uni=True)

  def set_effective_page_width(self, value):
    self.effective_page_width = value

  def set_effective_page_height(self, value):
    self.effective_page_height = value

  def set_multi_cell_width(self, value):
    self.multi_cell_width = value

  def set_rowh(self, value):
    self.rowh = value

  def first_page(self, langue):
    self.set_font(*self.font_2)
    self.set_text_color (*self.color_1)
    # Calculate width of title and position
    w = self.get_string_width(self.title_1) + 6
    self.set_x((self.WIDTH) / 2)    
    self.set_y((self.HEIGHT) /2.5)
    self.cell(0,0, self.title_1, 0, 0, 'C',0)
    self.ln(10)
    self.set_font(*self.font_1)
    self.cell(0,0, 'Français-'+self.local_language, 0, 0, 'C')
    self.add_page()
    self.set_text_color (0,0,0)

  def cover(self):
    self.image(self.cover_1,0,0,self.WIDTH,self.HEIGHT)
    self.set_text_color(*self.color_1)
    self.set_xy(self.l_margin, 90)
    self.set_font(*self.font_cover_1)
    self.cell(0,0, self.title_1, 0, 0, 'L')
    self.ln(18)  

    self.set_font(*self.font_cover_1)
    self.set_text_color (*self.color_2)
    self.cell(0,0, self.title_2, 0, 0, 'L')
    self.ln(28)

    self.set_font(*self.font_2)
    self.set_text_color (*self.color_2) 
    title = 'Français-'+self.local_language
    self.cell(0,0, title.upper(), 0, 0, 'L')

    self.image(self.logo_1,self.l_margin,self.HEIGHT-self.l_margin,30)

    self.add_page()  


In [31]:
def print_expression(langue, df, pdf, ind, chosen_font_expr, chosen_font_trad):
  # impression d'une ligne expression/traduction dans la bonne fonte
  ln = 6.5
  ybefore = pdf.get_y()
  pdf.set_font(chosen_font_expr, '', 10.0)
  pdf.set_text_color (0,0,0)
  pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, df['expression'][ind])
  pdf.set_xy(pdf.effective_page_width/2 + pdf.l_margin, ybefore)
  #---- cas de l'alphabet arabe
  pdf.set_font(chosen_font_trad, '', 10.0)
  if langue=='ams':
    arabic_string = arabic_reshaper.reshape(df[langue][ind])
    arabic_string = arabic_string[::-1]
    w = pdf.get_string_width(arabic_string) + 6
    pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, arabic_string, 0, 'R')
    #pdf.cell(w, 9, arabic_string, 0, 1, 'L', 0)
  else:
    pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, df[langue][ind])

  pdf.ln(ln)


In [32]:
def create_pdf_instance(item, df):
  pdf=PDF()
  pdf.set_langue (item['language'])
  pdf.add_fonts()
  pdf.format = 'A4'
  pdf.unit = 'in'
  pdf.set_margins(20.0, 10.0, 20.0)
  A4_height_inches = 11.6929
  effective_page_width = pdf.w - 2*pdf.l_margin
  effective_page_height = pdf.h - 2*pdf.b_margin
  multi_cell_width = (effective_page_width/2)-5
  pdf.set_effective_page_width(effective_page_width)
  pdf.set_effective_page_height(effective_page_height)
  pdf.set_multi_cell_width(multi_cell_width)

  rowh = 4.5
  pdf.set_rowh(rowh)
  ln = 6.5

  chosen_font_expr = 'DejaVuSans'
  # cut here -------------------------------------------------------------
  if item['trigramme']=='ams':
    # font arabe
    print("Font ams")
    chosen_font_trad = 'NotoSansArabic'
  elif item['trigramme']=='tig':
    # font amharique
    print("Font tig")
    chosen_font_trad='NotoSerifEthiopic'
  elif item['trigramme']=='arm':
    # font arménien
    print("Font arm")   
    chosen_font_trad = 'DejaVuSans' 
  else:
    # font standard
    chosen_font_trad = 'DejaVuSans'
    # amharique/tigrinya


  # Add new page. Without this you cannot create the document.
  pdf.add_page()
  # couverture
  pdf.cover()
  # Remember to always put one of these at least once.
  pdf.set_font('Times','',10.0) 


  pdf.first_page(item['language'])

  pdf.ln(ln)
  idx = '' # index alpha (start)
  # pour chaque élement expression/traduction
  for ind in df.index:
    ybefore = pdf.get_y()
    if df['index'][ind] != idx:
      # changement de lettre index
      # vérification place en bas suffisante
      space_left = effective_page_height-pdf.get_y()
      # changement de page si place restante insuffisante
      if space_left < 2:
        pdf.add_page()
        pdf.ln(ln*2)    

      pdf.ln(ln*1.5)
      ybefore = pdf.get_y()
      # changement lettre index
      pdf.set_font(*pdf.font_2)
      pdf.set_text_color (*pdf.color_4)
      pdf.multi_cell(multi_cell_width,  rowh, df['index'][ind])
      pdf.set_xy(effective_page_width/2 + pdf.l_margin, ybefore)
      pdf.multi_cell(multi_cell_width,  rowh, " ")
      idx = df['index'][ind]
      pdf.ln(ln)

    # expression / traduction
    print_expression(item['trigramme'], df, pdf, ind, chosen_font_expr, chosen_font_trad)

    # changement de page si place restante insuffisante
    space_left = effective_page_height-pdf.get_y()
    if space_left < 0:
      pdf.add_page()
      pdf.ln(ln*2)
    #print (df['expression'][ind], str(space_left))

  pdf.ln(ln)  
  # cut here -------------------------------------------------------------

  print ("écriture de " + item['trigramme'] + '.pdf')
  pdf.output(item['trigramme'] + '.pdf', 'F')

In [33]:
df_corpus_langue=[] 
#ouverture de chaque onglet Corpus de travail dans un DF
for index, item in enumerate(vk_lang_dict):
  df_corpus_langue.append (charge_corpus_langue(df_corpus, item['trigramme']))
  create_pdf_instance(item, df_corpus_langue[index])


Font ams
écriture de ams.pdf


/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65149
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65161
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65160
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65162
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65090
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65163
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.7/dist-packages/fpdf/tt

écriture de tur.pdf
écriture de rus.pdf
écriture de ukr.pdf
écriture de rou.pdf
écriture de hun.pdf
Font tig
écriture de tig.pdf
écriture de alb.pdf
écriture de geo.pdf
Font arm
écriture de arm.pdf
écriture de dar.pdf
écriture de pst.pdf
écriture de prs.pdf
écriture de aze.pdf
écriture de esp.pdf
écriture de amh.pdf
écriture de fma.pdf


In [34]:
! ls /usr/share/fonts/opentype/bebas-neue/

ls: cannot access '/usr/share/fonts/opentype/bebas-neue/': No such file or directory


## Classe de construction de PDF

In [ ]:
# Create instance of FPDF class
# Letter size paper, use inches as unit of measure
# https://stackoverflow.com/questions/58801358/displaying-arabic-words-incorrectly-in-fpdf-python

pdf=PDF()
pdf.set_langue ('ukr')
pdf.add_font('DejaVuSans', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
pdf.add_font('DejaVuSans-Bold', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
pdf.set_font('DejaVuSans', '', 9.0)
# amharique/tigrinya
pdf.add_font('NotoSerifEthiopic', '', '/usr/share/fonts/truetype/noto/NotoSerifEthiopic-Regular.ttf', uni=True)
#pdf.set_font('NotoSerifEthiopic', '', 9.0)

pdf.format = 'A4'
pdf.unit = 'in'
pdf.set_margins(20.0, 10.0, 20.0)
# Add new page. Without this you cannot create the document.
pdf.add_page()
 
# Remember to always put one of these at least once.
#pdf.set_font('Times','',10.0) 

A4_height_inches = 11.6929
effective_page_width = pdf.w - 2*pdf.l_margin
effective_page_height = pdf.h - 2*pdf.b_margin

multi_cell_width = (effective_page_width/2)-5

rowh = 4.5
ln = 6.5
pdf.ln(ln)
idx = '' # index alpha (start)
for ind in df.index:
  ybefore = pdf.get_y()
  if df['index'][ind] != idx:
    pdf.ln(ln*1.5)
    ybefore = pdf.get_y()
    # changement lettre index
    pdf.set_font('DejaVuSans-Bold','',24.0)
    pdf.set_text_color (98,163,98)
    pdf.multi_cell(multi_cell_width,  rowh, df['index'][ind])
    pdf.set_xy(effective_page_width/2 + pdf.l_margin, ybefore)
    pdf.multi_cell(multi_cell_width,  rowh, " ")
    idx = df['index'][ind]
    pdf.ln(ln)

  ybefore = pdf.get_y()
  pdf.set_font('DejaVuSans', '', 9.0)
  pdf.set_text_color (0,0,0)
  pdf.multi_cell(multi_cell_width,  rowh, df['expression'][ind])
  pdf.set_xy(effective_page_width/2 + pdf.l_margin, ybefore)
  #---- cas de l'alphabet arabe
  # arabic_string = arabic_reshaper.reshape(df['ams'][ind])
  # arabic_string = arabic_string[::-1]
  # w = pdf.get_string_width(arabic_string) + 6
  #pdf.cell(w, 9, arabic_string, 0, 1, 'L', 0)

  #pdf.set_font('NotoSerifEthiopic', '', 9.0)
  pdf.multi_cell(multi_cell_width,  rowh, df['rus'][ind])

  pdf.ln(ln)

  space_left = effective_page_height-pdf.get_y()
  if space_left < 0:
    pdf.add_page()
    pdf.ln(ln*2)
  #print (df['expression'][ind], str(space_left))

pdf.ln(ln)  
pdf.output('tut58.pdf', 'F')

''

In [ ]:
ls -l /usr/share/fonts/truetype/dejavu/

total 2956
-rw-r--r-- 1 root root 705684 Jul 30  2016 DejaVuSans-Bold.ttf
-rw-r--r-- 1 root root    549 Jun 14 08:39 DejaVuSans.cw127.pkl
-rw-r--r-- 1 root root 331992 Jul 30  2016 DejaVuSansMono-Bold.ttf
-rw-r--r-- 1 root root 340712 Jul 30  2016 DejaVuSansMono.ttf
-rw-r--r-- 1 root root 136927 Jun 14 08:39 DejaVuSans.pkl
-rw-r--r-- 1 root root 757076 Jul 30  2016 DejaVuSans.ttf
-rw-r--r-- 1 root root 356088 Jul 30  2016 DejaVuSerif-Bold.ttf
-rw-r--r-- 1 root root 380132 Jul 30  2016 DejaVuSerif.ttf


In [ ]:
# Import FPDF class
from fpdf import FPDF


# Create instance of FPDF class
# Letter size paper, use inches as unit of measure
pdf=FPDF(format='A4', unit='in')
pdf.add_font('DejaVuSans', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
pdf.set_font('DejaVuSans', '', 9.0)

pdf.add_font('NotoSerifEthiopic', '', '/usr/share/fonts/truetype/noto/NotoSerifEthiopic-Regular.ttf', uni=True)
pdf.set_font('NotoSerifEthiopic', '', 9.0)
# Add new page. Without this you cannot create the document.
pdf.add_page()
 
# Remember to always put one of these at least once.
#pdf.set_font('Times','',10.0) 

A4_height_inches = 11.6929
effective_page_width = pdf.w - 2*pdf.l_margin
effective_page_height = pdf.h - 2*pdf.b_margin

for ind in df.index:
  ybefore = pdf.get_y()
  pdf.multi_cell(effective_page_width/2,  0.15, df['expression'][ind])
  pdf.set_xy(effective_page_width/2 + pdf.l_margin, ybefore)
  pdf.multi_cell(effective_page_width/2,  0.15, df['tig'][ind])
  pdf.ln(0.15)
  space_left = effective_page_height-pdf.get_y()
  if space_left < 0:
    pdf.add_page()
  #print (df['expression'][ind], str(space_left))

pdf.output('test4.pdf','F')

''